# 3DSlicer .fcsv to coordinate.json file format
### At this current time, there is no organization system proposed by the Brain Imaging Data Structure (BIDS) community for how to organize anatomical fiducials or landmarks that are not associated with magnetoencephalography (MEG), electroencephalography (EEG), and intracranial electroencephalography (iEEG). Anatomical fiducials can have a variety of uses like the ones we highlight in the anatomical fiducial (AFID) placement protocol but also other general applications like annotating lesions or artifacts and centroids of structures. In this code, we use inspiration from the Brain Imaging Data Structure v1.7.0 (Appendix VIII: Coordinate systems) and create a way to communicate our coordiante data. 


In [ ]:
import os
import pandas as pd
import json
from glob import glob
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/ataha24/afids-data

Cloning into 'afids-data'...
remote: Enumerating objects: 1138, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 1138 (delta 0), reused 7 (delta 0), pack-reused 1128
Receiving objects: 100% (1138/1138), 172.51 MiB | 21.33 MiB/s, done.
Resolving deltas: 100% (866/866), done.


In [ ]:
# path to the bids folder
fcsv_files = sorted(glob('afids-data/data/templates/PD-25/*.fcsv')) #path to rater placements 

for file in fcsv_files:
  fid_data = pd.read_csv(file, skiprows=3, header=None)
  fid_data.rename(columns={0: 'node_id', 1: 'x', 2: 'y', 3: 'z', 4: 'ow', 5: 'ox',
                                     6: 'oy', 7: 'oz', 8: 'vis', 9: 'sel', 10: 'lock',
                                     11: 'label', 12: 'description', 13: 'associatedNodeID'}, inplace=True)
  afid_desc = {}
  for ifid in fid_data['description']:
      afid_desc[ifid] = fid_data[fid_data['description'] == ifid][['x', 'y', 'z']].values[0].tolist()
      #IntendedFor has to be changed to represent the markups or MRI file for which the .json coordinates file is for 
      afid_dict = {"IntendedFor": f"anat/*_space-T1w_desc-groundtruth_afids.fcsv" , "FiducialsCoordinateSystem": "RAS",
                                 "FiducialsCoordinateUnits": "mm",
                                 "FiducialsCoordinateSystemDescription": "RAS orientation: Origin halfway between LPA and RPA, positive x-axis towards RPA, positive y-axis orthogonal to x-axis through Nasion,  z-axis orthogonal to xy-plane, pointing in superior direction.",
                                 "FiducialsCoordinates": afid_desc}

      json_output = json.dumps(afid_dict, indent=4)
      with open(f"{file[0:-5]}_coordsystem.json", 'w') as fid:
                        fid.write(json_output)
                        fid.write('\n')